# Introduction to Markov Networks



## CSCI E-83
## Stephen Elston

in the previous lesson we explored directed Bayes networks (BNs). BNs are directed acyclic graphs (DAGs) which can represent causality and independencies of many probability distributions. However, there is no unique representation of independencies. Many BNs can exhibit the same independency structure, but with different causal relationships. 

Now, we will turn our attention to another class of probabilistic graphical models known as **Markov networks** (**MNs**) or **Markov graphical models**, or **Markov random fields**. Markov graphic models are not directed graphs. Yet, like BNs, Markov graphical models can represent independencies of many probability distributions. 

As you will see, for certain distributions there are methods to map from BNs to Markov networks and back. However, this is not universally the case, and there are independencies which can only be represented by a MN, but not a BN

Like the directed Bayesian graphical models or DAGs, Markov networks provide a alternative **representation** method for probability distributions. In some cases, a Markov network represents the same independencies as the DAG, however in other cases, the independencies represented are not the same. The representation within an intelligent agent is illustrated schematically in the figure below. 

<img src="img/Representation.JPG" alt="Drawing" style="width:400px; height:200px"/>
<center> **Representation in an intelligent agent** </center>


## Relationship between DAGs and Markov networks  

As you have seen, the dependencies of some probability distributions can be represented by the d-separation criteria on DAGs or directed Baysian graphical models. Further, by their very nature, DAGs model causality. As a result of these properties DAGs represent conditional probability distributions.  

In contrast, Markov networks are undirected graphs. As you will see, they can model independencies through another type of separation criteria. However, Markov networks model **potentials**, which are not probability distributions at all. You can think of Markov networks as representing correlations. 

## Graph separation criteria

You can develop a useful view of the relationship between BNs and MNs by transformming a directed graph into a MN. First, we need to introduce some definitions:  

> **Definition:** An **imorality** in a directed graph $G$ occurs where either; a) there is a directed edge between $X$ and $Y$, or b) $X$ and $Y$ are both parents of the same note $Z$. 

This leads to a concept of a **moralized graph** that relates a directed BN to an undirected MN:

> **Definition:** A **moral graph**, $M(G)$, of a BN structure, $G$ is the **undirected graph** over $X$ that contains an undirected edge between $X$ and $Y$ if; a) there is a directed edge between $X$ and $Y$, or b) $X$ and $Y$ are both parents of the same note $Z$.

This leads us to a corollary of relating the independencies of the directed BN to the independencies of a MN:

> **Corollary:** Given a distribution $P_B$ such that $B$ is a parameterization on a graph $G$, then $M(G)$ is an I-map for $P_B$.

What does all of this mean? In summary, we can create an MN representing an independency structure starting with a DAG. However, this does not mean that the independency structure will be the same. The example in the Figure below will help to illustrate this idea. 

<img src="img/MoralizedGraph.JPG" alt="Drawing" style="width:600px; height:200px"/>
<center> **Example of Graph Moralization** </center>

Referring to the above illustration: 
1. The original DAG is shown on the left. The variable Z is the evidence variable. Z is said to **block** the path from X to W. 
2. The undirected **ancestral** graph is shown in the center. Since W has a blocked path in the DAG it is not part of the ancestral graph. U, X, Y and Z are all ancestors of W, and form the ancestral graph. 
3. We can make the following assertion about the DAG; $P(X,U\ |\ Z) = P(X\ |\ Z)\ P( U\ |\ Z)$.  This situation is an **immorality** since X and U are parents of Z. In other words, X and U are independent given evidence Z, which **blocks** the path from X and U. The graph is **moralized** by adding an edge between X and U as shown on the right. 

Given the two definitions and the corollary, in this case, the resulting moralized undirected graph has the same independencies as the original DAG. **If the DAG is an I-map of the distribution the undirected graph will also be an I-map. ???????????????** 

The conditional independencies in the moralied graph can be thought of as **separating** subsets of the Markov network. This leads to the following definition:

> **Definition:** Given subsets $X$, $Y$ and $Z$, $X$ and $Y$ are conditionally independent or **D-separated** conditioned on the subset $Z$ if they are separated on the moralized graph. 

The definition leads to another related definition:

> **Definition:** A graph $G$ is a **dependency map** or **D-map** of a distribution $P$ if the graph contains every conditional independence in $P$. We can represent this relationship as:

$$(X\ \bot\ Y\ |\ Z_G) \Leftarrow (X\ \bot\ Y\ |\ Z_P)$$

Recalling the definition of an I-map we can create another definition.

> **Definition:** If a graph $G$ is both an I-map and a D-map of a distribution $P$ we say that $G$ is a **prefect map** of $P$. We can write this relationships as:

$$(X\ \bot\ Y\ |\ Z_G) \Leftrightarrow (X\ \bot\ Y\ |\ Z_P)$$

> **Note:** While it would be nice if a graph were a perfect map of a distribution, this will rarely be the case in real world problems. Thus, a perfect map is mostly useful as a reference point in developing probabilistic graphical models.  

## Potentials and the Hammerly-Clifford theorem

In undirected Markov networks we associate maximal cliques with **potential functions** or **factors**. 

**Hammerly-Clifford theorem:** Let $p(x)$ be a strictly positive distribution and let $G$ be an undirected graph, the **conditional independence properties** of $p(x)$ are satisfied if and only if the distribution can be represented as a product of factors, one factor representing each maximal clique, $c$, of $G$:

$$p(x\ |\ \Theta) = \frac{1}{Z(\theta)} \prod_{c \in G} \psi_c (x_c\ | \theta_c)\\
where\\
Z(\theta) = \sum_x \prod_{c \in G} \psi_c (x_c\ | \theta_c)$$

Here $Z(\theta)$ is a partition function that ensures the distribution is in the range $\{0,1 \}$.

## Potentials and the Gibbs distribution

An **energy based model** helps us understand some of the deep connections between statistical mechanics and the potentials of MNs using the **Gibbs distribution**:

$$p(x\ |\ \theta) = \frac{1}{Z(\theta)} exp\big( - \sum_c E(x_c\ |\ \theta_c) \big)\\
where\\
E(x_c) = Energy\ of\ clique\ c$$

We can use the above result to represent a potential for a clique as:

$$\psi(x_c\ |\ \theta_c) = exp\big( -E(x_c\ |\ \theta_c) \big)$$

We can interpret this representation as meaning that **high probability states** of the clique, $c$, have **low energy**. 

A a distribution, $p$ that satisfies the conditional independence properties of the Markov undirected graph, $G$, the conditional probability can be represented as the normalized product of potentials:

$$p(x\ |\ \theta) = \frac{1}{Z(\theta)} \prod_c \psi(x_c\ |\ \theta_c) \\
where\\
Z(\theta) = \sum_x \prod_c \psi(x_c\ |\ \theta_c) $$

In summary, potentials give us a method to represent conditional distributions by undirected graphs, given the correct independencies. 

## Representation with MNs

The foregoing has all be rather abstract. Let's try simple example. For this example, we will factor a probability distribution on a graphical model into potentials. We will use the student letter example we used it the previous lesson. The DAG representing the distribution, $P(I,D,G,S,L)$, is shown in the figure below.

<img src="img/LetterDAG.JPG" alt="Drawing" style="width:600px; height:400px"/>
<center> **DAG for student letter and GRE score** </center>

Referring to the above figure, we can factorize the directed graph or BH as conditional probabilities:

$$P(I,D,G,S,L) = P(I)\ P(D)\ P(S\ |\ I)\ P(G\ |\ I, D)\ P(L\ |\ G)$$

Notice that the **directed edges** of the graph provide **causal relationships**. We will loose this causality when we transform the representation to an undirected graph. 

With **undirected edges** we must model the **correlations** between the variables. We factorize the distribution on the MN using potentials. There is a potential for each **clique** on the **moralized** undirected graph. As a first step we need to create the undirected graph and moralize it:

<img src="img/MoralizedLetter.JPG" alt="Drawing" style="width:600px; height:350px"/>
<center> **Transforming DAG to Moralized MN** </center>

The graph is now undirected. The addition of the edge between $I$ and $D$ moralizes this graph.

## Do DAGs and Markov networks have equivalent representations?

We have seen that we can transform a DAG into a Markov networks. The question should be, are these representations truly equivalent? In general, the answer is no. As we have seen, the Markov network contains additional links from the moralization of the directed graph. As a result, independence information can be lost. For example, consider a distribution $p(Z|X,Y)\ p(X)\ p(Y)$ represented by a Markov network for the with a single clique $\phi(X,Y,Z)$. This representation does not allow us to infer the relationship $X\ \bot\ Y$.   

To address the question if we can create a DAG that can represent any Markov network, consider an example with the potentials $\phi(W,X)\ \phi(X,Y)\ \phi(Y,Z)\ \phi(Z,W)$. There is no singly connected acyclic graph that can represent this relationship. Again, there is no equivalent representation.    

From the above, you can see that there are limitations to the representations possible with either DAGs or Markov networks. 

## Cliques on undirected graphs

In order to factorize an undirected graph we must first decompose it into cliques. We can define a clique as follows:

> **Definition:** A **clique** is a fully connected set of neighbors on an undirected graph. 

Let's apply this definition to the example in the figure below. 

<img src="img/Cliques1.JPG" alt="Drawing" style="width:600px; height:250px"/>
<center> **Cliques of a Chain Markov Network** </center>

On the left side of the figure is a simple **chain** MN. There are only two possible **fully connected** cliques. These two cliques are $\{1,2\}$ and $\{2,3\}$ as illustrated above.

The above example is simple, unrealistically so. The figure below shows a somewhat more complex undirected graph. 

<img src="img/Cliques2.JPG" alt="Drawing" style="width:600px; height:250px"/>
<center> **Cliques of a Undirected Graph** </center>

The MN in the above diagram has two fully connected cliques; $\{1,2,3\}$ and $\{2, 3, 4\}$. The sets $\{1,2\}$, $\{2,3\}$, $\{3,4\}$, and $\{1,4\}$ cannot be cliques as each node has other connections that will form fully connected cliques. Further, $\{1,2,4\}$ and $\{1, 3, 4\}$ cannot be cliques as they are not fully connected. 


## Factorization on Markov networks

Given this definition it is easy to find the cliques of our MN as illustrated in the figure below:

<img src="img/LetterCliques.JPG" alt="Drawing" style="width:600px; height:400px"/>
<center> **Cliques of the Undirected Markov Network** </center>

The undirected graph has three cliques; {D,I,G}, {I,S} and {G,L}. Now, we are in a position to factorize the MN into a set of clique potentials:

$$P(I,D,G,S,L) = \frac{1}{Z} exp\{ E(D,I,G) + E(G,L) + E(I,S) \} \\
= \frac{1}{Z} \psi(D,I,G)\ \psi(G,L)\ \psi(I,S)$$

Where    
$E() = $ **expectation** function,  
$\psi() = $ **clique potential**, and
$Z = $ **Partition function**:

$$Z = \sum_{I,D,G,S,L} \psi(D,I,G)\ \psi(G,L)\ \psi(I,S)$$

To summarize, the joint probability distribution is modeled as the product of potentials of the undirected graph or MN.  

## Summary of Bayesian network semantics  

We have seen how distributions can be factorized on Bayesian networks. We can make the following statements:

1. Factorization on BNs is based on conditional distributions.  
2. Factorization on a BN implies conditional independencies.   

These facts lead to the following definition:

> **Definition:** Given a distribution $P$ and $G$ is a Bayesian network, $P$ factorizes as a set of CDPs specified as the nodes of $G$. 

## Markov properties and separation in undirected graphs

 > **Definition:** For a graph $G$, **disjoint** subsets $A$ and $B$ are separated by subset $S$ if every path from $A$ to $B$ passes through $S$ then $S$ **separates** $A$ and $B$. Or, if $S = \emptyset$, then no path exists from $A$ to $B$, and $A$ and $B$ are **separated**. 
 
 > **Definition:** For a graph $G$ with disjoint sets $A$, $B$ and $S$, where $S$ separates $A$ and $B$, then $A\ \perp\ B\ \| S$.
 
This definition leads to the concepts of **soundness** and **completeness**. 

> **Theorem:** For any graph $G$ that factorizes a distribution $P$ then $I(G) \subseteq I(P)$. This relationship is known as the **soundness** property.  

> **Claim:** For any graph $G$, with subsets $X$, $Y$ and $Z$, that factorizes a distribution $P$, if  $(X \bot Y\ |\ Z) \subseteq I(P)$ the d-$sep_G(X;Y\ |\ Z)$. This relationship is known as the **completeness** property.

## Markov blanket

> **Definition:** A subset of $X$, $U$, is a **Markov blanket** in a distribution $P$ if $X \notin U$ and if, $U$, is a minimal set of nodes following the relationship: 

$$(X\ \bot\ \chi - \{X \} - U\ |\ U)\ \in\ I(P)$$.

The concept of a Markov blanket is illustrated in the figure below. 

<img src="img/MarkovBlanket.JPG" alt="Drawing" style="width:300px; height:300px"/>
<center> **Markov blanket of a node in a graph** </center>   

In the above illustration the nodes, $\{ 2, 6, 7 \}$ are the Markov blanket of node $4$. You can see that the Markov blanket is the minimum subset of nodes which influence the node $4$. Conversely, node $4$ can only influence nodes in its Markov blanket.     

In [1]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

In [2]:
student_model = BayesianModel([('D', 'G'), ('I', 'G'), ('G', 'L'), ('I', 'S')])

CDP_D = TabularCPD(variable='D', variable_card=2, values=[[0.7, 0.3]])
CDP_I = TabularCPD(variable='I', variable_card=2, values=[[0.8, 0.2]])
CDP_L = TabularCPD(variable='L', variable_card=2, 
                   values=[[0.1, 0.4, 0.99],
                           [0.9, 0.6, 0.01]],
                   evidence=['G'], # Leter depends on the grade
                   evidence_card=[3])
CDP_S = TabularCPD(variable='S', variable_card=2,
                   values=[[0.95, 0.2],
                           [0.05, 0.8]],
                   evidence=['I'], # GRE score depneds on intelligence
                   evidence_card=[2])
CDP_G = TabularCPD(variable='G', variable_card=3, 
                   values=[[0.3, 0.05, 0.9,  0.5],
                           [0.4, 0.25, 0.08, 0.3],
                           [0.3, 0.7,  0.02, 0.2]],
                  evidence=['I', 'D'],
                  evidence_card=[2, 2])
student_model.add_cpds(CDP_D, CDP_I, CDP_S, CDP_G, CDP_L)
student_model.check_model()

True

In [3]:
student_model.get_immoralities()

{('D', 'I')}

In [5]:
student_jt = student_model.to_junction_tree()

C:\Users\StevePC2\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:586: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]


In [7]:
student_jt.get_factors()

[<DiscreteFactor representing phi(I:2, S:2) at 0x1adeae18a58>,
 <DiscreteFactor representing phi(I:2, G:3, D:2) at 0x1adeae18da0>,
 <DiscreteFactor representing phi(L:2, G:3) at 0x1adeae18e48>]

In [9]:
student_jt.marginalize(['L'])

AttributeError: 'JunctionTree' object has no attribute 'marginalize'

#### Copyright 2018, Stephen F Elston. All rights reserved. 